In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/comp90051-22-s2-authorship/sample.csv
/kaggle/input/comp90051-22-s2-authorship/train.json
/kaggle/input/comp90051-22-s2-authorship/test.json


In [35]:
train_data['authors'][0]

[42, 13720, 36]

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms

batch_size = 64

# Prepare train and test dataloaders
train_data = pd.read_json('../input/comp90051-22-s2-authorship/train.json')
test_data = pd.read_json('../input/comp90051-22-s2-authorship/test.json')
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True, drop_last = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, shuffle = True, drop_last = True)

In [5]:
print(train_loader.dataset.shape,test_loader.dataset.shape)

(25793, 5) (800, 6)


In [30]:
import torch.nn as nn
import torch.nn.functional as F

HIDDEN_DIM1 = 200
# Build Neural Network
class MLP(nn.Module):
    def __init__(self, n_features, n_classes, hidden_dim1 = HIDDEN_DIM1):
        super(MLP, self).__init__()
        
        # Initisation of weight matrix and bias term as model parameters
        self.fc1 = nn.Linear(n_features, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, n_classes)
        
    def forward(self, x):
        """
        Forward pass for logistic regression.
        Input: Tensor x of shape [N,C,H,W] ([batch size, channels, height, width])
        Output: Logit W @ x + b
        """
        
        x = x.view(x.shape[0], -1) #Flatten into vector
        x = F.relu(self.fc1)
        return self.fc2(x)
        


In [31]:
n_features, n_classes = train_loader.dataset.shape[0], 21246
logistic_regression = MLP(n_features, n_classes)

for p in logistic_regression.parameters():
    print(p.shape)



torch.Size([200, 25793])
torch.Size([200])
torch.Size([21246, 200])
torch.Size([21246])


In [27]:
def train(model, train_loader, test_loader, optimizer, n_epoch = 10):
    criterion = nn.CrossEntropyLoss() #cross entropy loss
#     predictions = []
    for epoch in range(n_epoch):
        for i, data in enumerate(train_loader,0):
            x, labels = data
            
            outputs = model(x)
#             predicted = torch.argmax(outputs, dim = 1)
#             predictions.append(outputs)
            optimizer.zero_grad() #Zero gradients
            loss = criterion(outputs, labels)
            loss.backward() #Compute parameter gradients using autograd
            optimizer.step() #Update weight parameter 
            
            


In [28]:
optimizer = torch.optim.Adam(logistic_regression.parameters(), lr = 3e-4)
train(logistic_regression, train_loader, test_loader, optimizer)

KeyError: 24022